In [1]:
import torch
from math import pi
import einops as einops

from training.utils.FLAIR_2 import*
import matplotlib.pyplot as plt
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

/Users/hugoriffaud/Documents/Atomiser_CloudDetection/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


dm = CloudSEN12DataModule(batch_size=8, num_workers=8)
dm.setup("fit")

tmp_x=None
tmp_y=None

for x, y in dm.train_dataloader():
    tmp_x=x
    tmp_y=y
    break

/Users/hugoriffaud/Documents/Atomiser_CloudDetection/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [3]:
dm = Cloud95DataModule(batch_size=8, num_workers=0)
dm.setup("test")

for x, y in dm.test_dataloader():
    print(x.shape, y.shape)  # Expected: [B, 4, H, W], [B, H, W]
    break

torch.Size([8, 4, 384, 384]) torch.Size([8, 384, 384])


In [ ]:
from training.perceiver import *
from training.utils import *
from training.losses import *

from training.VIT import *
from training.ResNet import *
from collections import defaultdict
from training import *
import os
from pytorch_lightning import Trainer,seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint,GradientAccumulationScheduler
import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce
from pytorch_lightning.profilers import AdvancedProfiler
profiler = AdvancedProfiler(dirpath="./profiling", filename="advanced_profile.txt")
import matplotlib.pyplot as plt

seed_everything(42, workers=True)



seed_everything(42, workers=True)

torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')

config_model = read_yaml("./training/configs/Unet.yaml")
xp_name="testset"


wand = True
wandb_logger = None
if wand:
    if os.environ.get("LOCAL_RANK", "0") == "0":
        import wandb
        wandb.init(
            name=config_model['encoder'],
            project="Atomizer_BigEarthNet",
            config=config_model
        )
        wandb_logger = WandbLogger(project="Atomizer_Cloud")

#def __init__(self, config, wand, name)
model = Model(config_model,wand=wand,transform=None)

data_module=CloudSEN12DataModule(batch_size=config_model["dataset"]["batchsize"], num_workers=8)
# Callbacks
checkpoint_callback_IoU = ModelCheckpoint(
    dirpath="./checkpoints/",
    filename=config_model["encoder"]+str(xp_name)+"-alone_best_model_val_mod_val-{epoch:02d}-{val_mod_val_ap:.4f}",
    monitor="val_IoU",
    mode="max",
    save_top_k=1,
    verbose=True,
)






accumulator = GradientAccumulationScheduler(scheduling={0: 64,50:8})

# Trainer
trainer = Trainer(
    #strategy="ddp",
    #strategy='ddp_find_unused_parameters_true',
    devices=-1, 
    max_epochs=config_model["trainer"]["epochs"],
    logger=wandb_logger,
    log_every_n_steps=16,
    accelerator="gpu",
    callbacks=[checkpoint_callback_IoU],#,accumulator],
    default_root_dir="./checkpoints/",
    #val_check_interval=0.3,
    precision="bf16-mixed",
    profiler=profiler,
    limit_test_batches=2,
    limit_train_batches=2,
    limit_val_batches=2
)


# Fit the model
trainer.fit(model, datamodule=data_module)






# ... after training completes, within your "if wand" block:
if wand and os.environ.get("LOCAL_RANK", "0") == "0":
    run_id = wandb.run.id
    print("WANDB_RUN_ID:", run_id)
    
    # Create the directory for storing wandb run IDs if it doesn't exist
    runs_dir = "training/wandb_runs"
    os.makedirs(runs_dir, exist_ok=True)
    
    
    # Save the run ID to a file inside wandb_runs
    run_file = os.path.join(runs_dir, xp_name+".txt")
    with open(run_file, "w") as f:
        f.write(run_id)








Seed set to 42
Seed set to 42


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁
log train_loss,▁
log val_loss,▁
train_loss,▁
trainer/global_step,▁▁
val_Dice,▁
val_IoU,▁
val_Precision,▁
val_Recall,▁
val_loss,▁
epoch,0


/Users/hugoriffaud/Documents/Atomiser_CloudDetection/training/Unet/unet.py:211: FutureWarning: `nn.init.xavier_normal` is now deprecated in favor of `nn.init.xavier_normal_`.
  init.xavier_normal(m.weight)
/Users/hugoriffaud/Documents/Atomiser_CloudDetection/training/Unet/unet.py:212: FutureWarning: `nn.init.constant` is now deprecated in favor of `nn.init.constant_`.
  init.constant(m.bias, 0)
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/hugoriffaud/Documents/Atomiser_CloudDetection/venv/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/Users/hugoriffaud/Documents/Atomiser_CloudDetection/venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_che

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/hugoriffaud/Documents/Atomiser_CloudDetection/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/hugoriffaud/Documents/Atomiser_CloudDetection/venv/lib/python3.11/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 0: 100%|██████████| 2/2 [00:24<00:00,  0.08it/s, v_num=i2e4]         

Python(85208) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85209) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85210) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85211) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85212) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85213) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85214) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(85215) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
